In [1]:
import sys
sys.path.append('/home/thomas/RAFT')
sys.path.append('/home/thomas/RAFT/core')
sys.path.append('/home/thomas/RAFT/models')

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time
import cv2
from PIL import Image as IMG
import time
import random
import torch.optim as optim
import zipfile
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
import rospy
from scipy.spatial.transform import Rotation as R
from message_filters import Subscriber, ApproximateTimeSynchronizer
import torch.nn.init as init
from copy import deepcopy
import torchvision.models as models
import itertools
import os

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import torchvision.transforms as T

In [4]:
def add_specular_highlight(img, intensity=1.5, radius=20):
    """Adds a bright circular spot to simulate reflection on glass/shiny surfaces."""
    h, w, _ = img.shape
    cx, cy = random.randint(0, w-1), random.randint(0, h-1)
    mask = np.zeros((h, w), dtype=np.float32)
    cv2.circle(mask, (cx, cy), radius, 1, -1)
    mask = cv2.GaussianBlur(mask, (radius//2*2+1, radius//2*2+1), 0)
    highlight = (img * (1 + intensity * mask[..., None])).clip(0, 1)
    return highlight

def add_lens_flare(img, num_streaks=3):
    """Adds simple lens flare streaks."""
    h, w, _ = img.shape
    flare = img.copy()
    cx, cy = random.randint(0, w-1), random.randint(0, h-1)
    for _ in range(num_streaks):
        dx, dy = random.choice([(-1,0),(1,0),(0,-1),(0,1)])
        length = random.randint(50, min(h,w)//2)
        overlay = np.zeros_like(img)
        for i in range(length):
            x, y = cx + dx*i, cy + dy*i
            if 0 <= x < w and 0 <= y < h:
                overlay[y, x] = [1, 1, 1]  # white flare
        flare = np.clip(flare + cv2.GaussianBlur(overlay, (15,15), 0)*0.3, 0, 1)
    return flare

def add_tint(img, color=(0.8, 1.0, 1.2)):
    """Applies a global tint (RGB multiplier)."""
    tinted = (img * np.array(color)[None, None, :]).clip(0, 1)
    return tinted

def pt_trans(img, ph_trans=False, homo=False):
    """
    img: numpy array of shape (H, W), grayscale image
    ph_trans: whether to apply photometric transforms
    """
    if ph_trans:
        if img.ndim == 2:
            img = np.stack([img]*3, axis=-1)
        img = torch.tensor(img).permute(2,0,1) / 255.0
        img = T.ToPILImage()(img)
        photometric_transforms = T.Compose([
            T.ColorJitter(
                brightness=0.5,
                contrast=0.5,
                saturation=0.5,
                hue=0.1
            ),
            T.RandomApply([T.GaussianBlur(3)], p=0.2),
        ])
        img = photometric_transforms(img)
        img = T.ToTensor()(img).permute(1,2,0).numpy()
        if random.random() < 0.45:
            noise = np.random.normal(0, 0.05, img.shape)
            img = np.clip(img + noise, 0, 1)
        if random.random() < 0.45:
            img = add_specular_highlight(img, intensity=1.5, radius=random.randint(10,30))
        if random.random() < 0.45:
            img = add_lens_flare(img, num_streaks=2)
        if random.random() < 0.45:
            tint_color = tuple(0.7 + 0.6*random.random() for _ in range(3))
            img = add_tint(img, tint_color)

        img = (255 * img).astype(np.uint8)
        if not(homo):
            return img
        else:
            return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [5]:
class ModResNet2(nn.Module):
    def __init__(self, in_chans, out):
        super(ModResNet2, self).__init__()
        original_model = models.resnet101(pretrained=True)
        
        original_model.conv1 = nn.Conv2d(
                    in_channels=in_chans,  # Change from 3 to 1 to accept grayscale images
                    out_channels=original_model.conv1.out_channels,
                    kernel_size=original_model.conv1.kernel_size,
                    stride=original_model.conv1.stride,
                    padding=original_model.conv1.padding,
                    bias=original_model.conv1.bias)
        
        self.features = nn.Sequential(
            original_model.conv1,
            original_model.bn1,
            original_model.relu,
            original_model.maxpool,
            original_model.layer1,
            original_model.layer2,
            original_model.layer3,
            original_model.layer4
        )
        self.avgpool = original_model.avgpool
        
        num_features = original_model.fc.in_features
        num_out_feas = out
        original_model.fc = nn.Linear(num_features, num_out_feas)
        self.fc = original_model.fc
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        out_fc = self.fc(x)
        return out_fc #out_conv#, 

In [6]:
class PoseTransformer2(nn.Module):
    def __init__(self, num_layers=12, embed_dim=512, num_heads=8, ff_dim=512, 
                 dropout=0.1, out_dim=512):
        super().__init__()  
        self.pos_embedding = nn.Parameter(torch.randn(1, 361, embed_dim))  # 19x19=361 tokens
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            activation='gelu',
            batch_first=True,
            norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc1 = nn.Linear(embed_dim, out_dim)
        self.embed_dim = embed_dim

    def forward(self, x):
        B = x.shape[0]
        x = x.view(B, self.embed_dim, -1).permute(0, 2, 1)  # (B, 361, embed_dim)
        # x = x + self.pos_embedding
        x = self.transformer(x)
        x = x.mean(dim=1)
        x = self.fc1(x)
        return x

In [7]:
class SiamesePoseNet3b_trans2d(nn.Module):
    def __init__(self):
        super(SiamesePoseNet3b_trans2d, self).__init__()
        self.model = ModResNet2(1,512)
        self.model2a = PoseTransformer2(embed_dim=512, out_dim=2, num_layers=2)
        self.model2b = PoseTransformer2(embed_dim=512, out_dim=2, num_layers=2)
        self.model2d = PoseTransformer2(embed_dim=512, out_dim=4, num_layers=2)
    def forward(self, rgbd1):
        f1_rgb = self.model(rgbd1[:,0:1,:,:])
        f2_rgb = self.model(rgbd1[:,1:,:,:])
        pfocal = self.model2a(f1_rgb - f2_rgb)
        pcenters = self.model2b(f1_rgb - f2_rgb)
        pquat = self.model2d(f1_rgb - f2_rgb)
        return pfocal, pcenters, pquat

In [94]:
def training_homo(p_gt, optimz, mse_loss_f, mod1, l1=1e4, l2=1.0):
    img_tsr, gt_out = p_gt
    gt_out = gt_out.to(device) 
    loss = 0.0
    pred_out = mod1(img_tsr.to(device))
    pscl, ptrs, pquat = pred_out
    pred_out_q = pquat/pquat.norm(p=2,dim=1,keepdim=True)
    print("pred_out_q: ", pred_out_q[0,:])
    print("gt_q_0: ", gt_out[0,4:])
    print("pscl: ", pscl[0,:])
    print("gt_scl: ", gt_out[0,0:2])
    print("ptrs: ", ptrs[0,:])
    print("gt_trs: ", gt_out[0,2:4])
    loss1 = l2*mse_loss_f(pscl, gt_out[:,:2])
    loss2 = l2*mse_loss_f(ptrs, gt_out[:,2:4])
    loss3 = l1*mse_loss_f(pred_out_q, gt_out[:,4:])
    print("loss1: ", loss1)
    print("loss2: ", loss2)
    print("loss3: ", loss3)
    loss = loss1 + loss2 + loss3
    if torch.is_tensor(loss):   
        optimz.zero_grad()
        loss.backward()
        optimz.step()
        
    with torch.no_grad():
        rotation_loses = [loss1, loss2, loss3, loss]
        pq = pred_out_q.cpu().numpy()
        pqq = np.vstack((pq[:,1], pq[:,2], pq[:,3], pq[:,0]))
        p_eul = np.array(R.from_quat(pqq.T).as_euler('xyz',degrees=True))
        gtq = gt_out[:,4:].cpu().numpy()
        gqq = np.vstack((gtq[:,1], gtq[:,2], gtq[:,3], gtq[:,0]))
        g_eul = np.array(R.from_quat(gqq.T).as_euler('xyz',degrees=True))
        
        out2, out3 = torch.hstack((pscl, ptrs)).cpu().numpy(), gt_out[:,0:4].cpu().numpy()
        out2 = np.hstack((out2, p_eul))
        out3 = np.hstack((out3, g_eul))
        
        for i, rot_loss in enumerate(rotation_loses):
            if torch.is_tensor(rot_loss):
                rotation_loses[i] = rot_loss.item()
            else:
                rotation_loses[i] = None
        return (out2, out3), rotation_loses

In [122]:
def training_opt(p_gt, optimz, mse_loss_f, mod2, l4=1.0, iters=5):
    img_tsr, uvc1  = p_gt
    p_del_uv = mod2(img_tsr[:,0:3,:,:].to(device), img_tsr[:,3:,:,:].to(device), iters=iters, test_mode=False)
    loss4 = 0.0
    all_gt_uv2 = []
    all_pd_uv2 = []
    for kp in range(len(uvc1)):
        u1c = uvc1[kp][:,0].long()
        v1c = uvc1[kp][:,1].long()
        for jp in range(len(p_del_uv)):
            pred_uv2 = p_del_uv[jp].permute(0,2,3,1)[kp, v1c, u1c,:] + uvc1[kp][:,0:2].to(device)  
            gt_uv2 = uvc1[kp][:,2:4]
            loss4 = loss4 + l4*mse_loss_f(pred_uv2, gt_uv2.to(device))
        with torch.no_grad():
            if kp<3:
                rind = np.random.randint(len(gt_uv2))
                print('pred_uv2: ', pred_uv2[rind,:].flatten())
                print('gt_uv2: ', gt_uv2[rind,:].flatten())
                
                vind = random.sample(range(len(gt_uv2)),100)
                all_gt_uv2.append(gt_uv2.numpy()[vind,:])
                all_pd_uv2.append(pred_uv2.cpu().numpy()[vind,:])
    print("loss4: ", loss4)
    loss = loss4
    if torch.is_tensor(loss):   
        optimz.zero_grad()
        loss.backward()
        optimz.step() 
    with torch.no_grad():
        rotation_loses = [loss.cpu().numpy().item()]
        return (all_pd_uv2, all_gt_uv2), rotation_loses

In [10]:
from utils import flow_viz
from utils import frame_utils
from raft import RAFT
from utils.utils import InputPadder, forward_interpolate
import argparse

In [ ]:
criterion1 = nn.SmoothL1Loss().to(device)
## Homography Alignment network
mod1 = SiamesePoseNet3b_trans2d().to(device)
mod1.load_state_dict(torch.load('./homo_match_comb_' +str(0)+ '.pth'))

## RAFT optical-flow network
parser = argparse.ArgumentParser()
parser.add_argument('--model', help="restore checkpoint")
parser.add_argument('--dataset', help="dataset for evaluation")
parser.add_argument('--small', action='store_true', help='use small model')
parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
parser.add_argument('--alternate_corr', action='store_true', help='use efficent correlation implementation')
args, unknown = parser.parse_known_args()
mod2 = torch.nn.DataParallel(RAFT(args))
mod2.load_state_dict(torch.load('./raft-things.pth'))
mod2 = mod2.cuda()

/home/thomas/anaconda3/envs/yolov8/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/thomas/anaconda3/envs/yolov8/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/thomas/anaconda3/envs/yolov8/lib/python3.9/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [70]:
def remap_homo(img1, Hp_1_2, ww=640, hh=480, mxlim=3500, inv=False):
    H, W = img1.shape[:2]
    corners = np.array([
        [0, 0, 1],
        [W-1, 0, 1],
        [W-1, H-1, 1],
        [0, H-1, 1]
    ], dtype=np.float32).T  
    warped = Hp_1_2 @ corners
    warped /= warped[2]

    min_x, min_y = np.min(warped[:2],axis=1)
    max_x, max_y = np.max(warped[:2],axis=1)
    shift_x = -min_x if min_x < 0 else 0
    shift_y = -min_y if min_y < 0 else 0

    T = np.array([[1, 0, shift_x],
                  [0, 1, shift_y],
                  [0, 0, 1]], dtype=np.float32)
    H_shifted = T @ Hp_1_2
    new_w = max_x + shift_x
    new_h = max_y + shift_y
    sx = ww / new_w
    sy = hh / new_h
    S = np.array([[sx, 0, 0],
                  [0, sy, 0],
                  [0, 0, 1]], dtype=np.float32)
    H_final = S @ H_shifted
    blk_img = cv2.warpPerspective(img1, H_final, (ww, hh))
    mod_params = (shift_x, shift_y, sx, sy)
    return blk_img, H_final, mod_params


In [74]:
def construct_homo(img, K1, K2, R1, R2, params, inv=False, Hp=None, dft=False):
    img = np.array(img).astype(np.uint8)
    h, w = img.shape[:2]
    sf, sc = params
    K2 = np.array(K2)
    K2[0,0] = sf[0] + K2[0,0]
    K2[1,1] = sf[1] + K2[1,1]
    K2[0,2] = sc[0] + K2[0,2]
    K2[1,2] = sc[1] + K2[1,2]
    mod_params = (0.0, 0.0, 1.0, 1.0)
    if inv:
        H2 = (K2@R2)@np.linalg.inv(K1@R1)
        Hp = H2@Hp
        Hinv = np.linalg.inv(Hp)
        Hinv /= Hinv[2, 2]
        rotated_image = cv2.warpPerspective(img, Hinv, (w, h))
        return rotated_image, Hp
    else:
        H_o = (K2@R2)@np.linalg.inv(K1@R1)
        H = H_o/H_o[2, 2]
        if dft:
            rotated_image = cv2.warpPerspective(img, H, (w, h))
        else:
            rotated_image, H_shifted, mod_params = remap_homo(img, H_o)
            H_o = np.array(H_shifted)

        Hf = H_o 
        return rotated_image, K2, R2, Hf, mod_params

In [17]:
#TODO
## 3D coordinates of KLB bridge obtained from BIM
bx_smb = [np.array([[-1.03, -10.92, 16.96],[-1.03,2.85,16.96],[-1.03,2.85,13.97],[-1.03,-10.66,13.97]]),
          np.array([[-1.03, 2.85, 16.96],[-1.03,16.62,16.96],[-1.03,16.36,13.97],[-1.03,2.85,13.97]]),
          np.array([[-1.03, 2.85, 13.97],[-1.03,16.36,13.97],[-1.03,16.1,10.97],[-1.03,2.85,10.97]]),
          np.array([[-1.03,-10.66, 13.97],[-1.03,2.85,13.97],[-1.03,2.85,10.97],[-1.03,-10.39,10.97]])]

In [18]:
#TODO
## The calibration parameters had already been obtained using INAF
## Define camera intrinsic matrix
K_mm = np.array([[508.3997, 0, 316.0652],[0,677.8663,254.0068],[0, 0, 1]])
nimg_0 = [13,19,22,21,48,65,52,54,55,31,32,7,14]
nimg_g = list(itertools.combinations(nimg_0, 2))
klb_dir = './smart_bridge2/'
IMGS, TRNS, QUTS = [], [], []
for idx in nimg_0:
    img_idx = cv2.imread(klb_dir+'img'+str(idx)+'.png')
    IMGS.append(cv2.cvtColor(img_idx,cv2.COLOR_BGR2RGB)) 
    trs = np.load(klb_dir+'trans_'+str(idx)+'.npy').flatten()
    qts = np.load(klb_dir+'quat_'+str(idx)+'.npy').flatten()
    TRNS.append(trs) 
    QUTS.append(qts)
 

In [19]:
def calc_ffbx(K_mm, Rts, xyz_0, trs, th=100):
    zuv0 = K_mm@Rts@(xyz_0.T - trs.reshape(-1,1))
    uv_0 = (zuv0[0:2,:]/zuv0[2:,:]).astype(np.int16)
    return uv_0

In [20]:
## Generation of patches
BXMs = []
n_patches = 4
for idx in nimg_0:
    trs = np.load(klb_dir+'trans_'+str(idx)+'.npy').flatten()
    qts = np.load(klb_dir+'quat_'+str(idx)+'.npy').flatten() #0.95
    Rts = np.array(R.from_quat([qts[1],qts[2],qts[3],qts[0]]).as_matrix())
    xyz_0 = np.array([[-1.03, -10.92, 16.96],[-1.03,2.85,16.96],[-1.03,2.85,13.97],[-1.03,-10.66,13.97]])
    xyz_1 = np.array([[-1.03, 2.85, 16.96],[-1.03,16.62,16.96],[-1.03,16.36,13.97],[-1.03,2.85,13.97]])
    xyz_2 = np.array([[-1.03, 2.85, 13.97],[-1.03,16.36,13.97],[-1.03,16.1,10.97],[-1.03,2.85,10.97]])
    xyz_3 = np.array([[-1.03,-10.66, 13.97],[-1.03,2.85,13.97],[-1.03,2.85,10.97],[-1.03,-10.39,10.97]])
    uv_0 = calc_ffbx(K_mm, Rts, xyz_0, trs)
    uv_1 = calc_ffbx(K_mm, Rts, xyz_1, trs)
    uv_2 = calc_ffbx(K_mm, Rts, xyz_2, trs)
    uv_3 = calc_ffbx(K_mm, Rts, xyz_3, trs)
    BXMs.append([uv_0.T, uv_1.T, uv_2.T, uv_3.T])

In [21]:
def comp_mskk(im, uv_1):
    im = np.array(im).astype(np.uint8)
    hh,ww = im.shape[0:2]
    msk = np.zeros((hh,ww),dtype=np.uint8)
    pts = np.array(uv_1[0:2,:]).T.astype(np.int32)
    cv2.fillPoly(msk, pts[np.newaxis,:,:], 1)
    if len(im.shape)>2:
        msk = np.stack([msk]*3,-1)
    imgi = msk*im
    return imgi

In [22]:
def fratio(uv_1, uv_2, th=0.2, w=640, h=480):
    uv_2[0,uv_2[0,:]<0] = 0
    uv_2[0,uv_2[0,:]>w-1] = w-1
    uv_2[1,uv_2[1,:]<0] = 0
    uv_2[1,uv_2[1,:]>h-1] = h-1
    vmin,vmax = np.nanmin(uv_2[1,:])/h,np.nanmax(uv_2[1,:])/h
    umin,umax = np.nanmin(uv_2[0,:])/w,np.nanmax(uv_2[0,:])/w
    a2 = (vmax-vmin)*(umax-umin)
    vmin1,vmax1 = np.nanmin(uv_1[1,:])/h,np.nanmax(uv_1[1,:])/h
    umin1,umax1 = np.nanmin(uv_1[0,:])/w,np.nanmax(uv_1[0,:])/w
    a1 = (vmax1-vmin1)*(umax1-umin1)
    flag = (a2/a1)>=th
    return flag

In [23]:
#TODO
#This holds for a 640x480 image frame - can be modified accordingly
U2v, V2v = np.arange(640),np.arange(480)
U2v, V2v = np.meshgrid(U2v, V2v)
U2v, V2v = U2v.flatten(), V2v.flatten()
onsv = np.ones_like(U2v)
U2v1 = np.vstack((U2v,V2v,onsv))

In [41]:
def synthetic_depth_surface(UV, curvature=0.0, max_depth_diff=1.0, center=None, coff=0.5):
    UV = np.asarray(UV)
    cid = np.random.randint(len(UV))
    center = UV[cid,:]
    d_norm = np.linalg.norm(UV - center, axis=1)
    rnd = random.choice([-1.0,1.0])
    surface = rnd * (np.exp(-curvature * d_norm**2) - 1.0)
    depth = surface * max_depth_diff + coff
    return depth.reshape(480,640)
    
def synthetic_trans3D(t1,R1,img1,K1,nnmax=1300):
    rnd = np.random.uniform(0,1.0,1)
    if rnd < 0.5:
        dt = np.random.uniform(-0.1,0.1,3).reshape(-1,1)
        dr = np.random.uniform(-5,5,3)
    elif (rnd>=0.5) and (rnd<0.7):
        dt = np.random.uniform(-0.5,0.5,3).reshape(-1,1)
        dr = np.random.uniform(-10,10,3)
    else:
        dt = np.random.uniform(-1.5,1.5,3).reshape(-1,1)
        dr = np.random.uniform(-15,15,3)
    max_d = np.random.uniform(1,3,1)
    cv = np.random.uniform(0.0,2.0,1) #np.random.uniform(-0.1,0.1)
    t2 = t1 + dt
    coff = t1[0,0] + np.random.uniform(-1.5,1.5,1)
    r1 = np.array(R.from_matrix(R1).as_euler('xyz',degrees=True))
    r2 = r1 + dr
    R2 = np.array(R.from_euler('xyz',r2,degrees=True).as_matrix())
    z2_0 = synthetic_depth_surface(U2v1[0:2,:].T, curvature=cv, max_depth_diff=max_d, coff=coff)
    z2 = z2_0[U2v1[1,:],U2v1[0,:]]
    z1u1 = K1@R1@(np.linalg.inv(K1@R2)@(z2*U2v1) + t2-t1)
    z1u1 = z1u1[0:2,:] / z1u1[2:,:]
    u1p, v1p = z1u1[0,:], z1u1[1,:]
    ffg = (u1p>=0) * (u1p<640) * (v1p>=0) * (v1p<480)
    u1p, v1p = u1p[ffg].astype(np.int16), v1p[ffg].astype(np.int16)
    if len(u1p)>=1000:
        u2p, v2p = U2v[ffg], V2v[ffg]
        blk_img = np.zeros_like(img1)
        blk_img[v2p,u2p,:] = img1[v1p,u1p,:]
        vv_2n, uu_2n, _ = np.where(blk_img > 0)
        if len(vv_2n)>=100*100:
            on2 = np.ones_like(vv_2n)
            U2v1_n = np.vstack((uu_2n, vv_2n, on2))
            z2_ = z2_0[vv_2n,uu_2n]
            z1u1n = K1@R1@(np.linalg.inv(K1@R2)@(z2_*U2v1_n) + t2-t1)
            z1u1n = z1u1n[0:2,:] / z1u1n[2:,:]
            u1pn, v1pn = z1u1n[0,:], z1u1n[1,:]
            ffg = (u1pn>=0) * (u1pn<640) * (v1pn>=0) * (v1pn<480)
            UV2n = np.vstack((uu_2n[ffg], vv_2n[ffg])).T
            UV1n = np.vstack((u1pn[ffg], v1pn[ffg])).T
            if len(UV1n)>nnmax:
                piddx = random.sample(range(len(UV1n)),nnmax)
                UV1n, UV2n = UV1n[piddx,:], UV2n[piddx,:]
            return blk_img, torch.tensor(UV1n), torch.tensor(UV2n)
        else:
            return False
    else:
        return False

In [108]:
def comb_imgs2(pid, pid2, imgi, T1_,R1_, params, K1, K2, R2, R3, sz=300, homo=False, viz=False):
    K2 = np.array(K2)
    K1 = np.array(K1)
    if not(homo):
        imgi = np.array(imgi)
    else:
        imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2GRAY).astype(np.uint8) 
    hhn, wwn = imgi.shape[0:2] 
    ptz_tsr_list = []
    ptz_opti_data = []
    ptz_tsr_list_viz = []
    mod_params = []
    bx_smb = BXMs[pid2]
    for kk, ptz in enumerate([bx_smb[pid]]):
        if ptz is not None:
            uv_ = ptz.T
            img1_pz = comp_mskk(imgi, uv_.astype(np.int16))
            imgiii = np.array(imgi).astype(np.uint8)
            rpt = np.random.uniform(0.0,1.0,1)
            if rpt < 0.25:
                ph_trans = True
                imgiii = pt_trans(imgiii, ph_trans, homo)
            img1_pz_ = comp_mskk(imgiii, uv_.astype(np.int16))
            FLG = False
            
            rn_opt = np.random.uniform(0,1)
            if not(homo) and rn_opt<0.3:
                osyn = synthetic_trans3D(T1_,R1_,img1_pz_,K1)
                if osyn:
                    img2_pz, coords1, coords2 = osyn
                    m_prms = (0.0, 0.0, 1.0, 1.0)
                    mod_params.append(m_prms)
                    FLG = True 
            else:
                rmd = random.choice([True, False])
                img2_pz, _, _, Hp, m_prms = construct_homo(img1_pz_, K1, K2, R2, R3, params, dft=rmd)
                ones_ = np.ones_like(uv_[0,:])
                uv_1 = np.vstack((uv_,ones_))
                uv_2 = Hp @ uv_1
                uv_2 = uv_2[0:2,:]/uv_2[2:,:]
                ohom = fratio(np.array(uv_), uv_2, th=0.4, w=wwn, h=hhn)
                if ohom:
                    mskk = np.zeros((hhn,wwn),dtype=np.uint8)
                    pts = np.array(uv_).T.astype(np.int32)
                    cv2.fillPoly(mskk, pts[np.newaxis,:,:], 1)
                    vv_1, uu_1 = np.where(mskk > 0)
                    ones_1 = np.ones_like(vv_1)
                    uv_op = np.vstack((uu_1, vv_1, ones_1))
                    uv_2op = Hp @ uv_op
                    uv_2op = uv_2op[0:2,:]/uv_2op[2:,:]
                    viz_flg = (uv_2op[0,:]>=0) * (uv_2op[0,:]<wwn) * (uv_2op[1,:]>=0) * (uv_2op[1,:]<hhn)
                    coords1 = torch.tensor(np.vstack((uu_1, vv_1)).T)[viz_flg,:]
                    coords2 = torch.tensor(uv_2op.T).float()[viz_flg,:]
                    mod_params.append(m_prms)
                    FLG = True
            if FLG:
                if len(coords1)>=100:
                    opti_data = torch.hstack((coords1,coords2))
                    ptz_opti_data.append(opti_data)

                    if not(homo):
                        img_tsr1 = torch.tensor(img1_pz).permute(2,0,1)
                        img_tsr2 = torch.tensor(img2_pz).permute(2,0,1)
                        comb_tsr = torch.concat([img_tsr1, img_tsr2],0).float()
                    else:
                        img_tsr1 = F.interpolate(torch.tensor(img1_pz/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                        img_tsr2 = F.interpolate(torch.tensor(img2_pz/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                        comb_tsr = torch.stack([img_tsr1, img_tsr2],0).float()

                    ptz_tsr_list.append(comb_tsr)
                    if viz:
                        if homo:
                            vis = np.hstack((img1_pz, img2_pz))
                        else:
                            h1, w1 = img1_pz.shape[:2]; h2, w2 = img2_pz.shape[:2]
                            vis = np.zeros((max(h1,h2), w1+w2, 3), dtype=np.uint8) 
                            vis[:h1, :w1, :] = img1_pz
                            vis[:h2, w1:w1+w2, :] = img2_pz
                            pts1 = coords1.numpy()
                            pts2 = coords2.numpy()
                            npmax = 10
                            if len(pts1)>npmax:
                                nnid = random.sample(range(len(pts1)),npmax)
                                pts1_ = pts1[nnid,:]
                                pts2_ = pts2[nnid,:]
                                for (x1,y1), (x2,y2) in zip(pts1_, pts2_):
                                    color = (0,255,0)
                                    pt1 = (int(x1), int(y1))
                                    pt2 = (int(x2)+w1, int(y2))
                                    cv2.line(vis, pt1, pt2, color, 1, cv2.LINE_AA)
                                    cv2.circle(vis, pt1, 4, color, -1)
                                    cv2.circle(vis, pt2, 4, color, -1)

                        ptz_tsr_list_viz.append(vis)
    return ptz_tsr_list, ptz_tsr_list_viz, ptz_opti_data, mod_params

In [76]:
def online_datagen(pid, Kmm,  sz=300, viz=False, nrep=30, homo=False):
    all_dat = []
    img12_tsr_all = []
    opt_data_all = []
    imgs_samps = []
    vizxx = []

    K1 = np.array(Kmm)
    K1_ = np.array(K1)
    K1 = np.array(K1)
    ofx, ofy, ocx, ocy = K1[0,0],K1[1,1],K1[0,2],K1[1,2]
    ofs = np.array([ofx,ofy]).flatten()
    ocs = np.array([ocx,ocy]).flatten()
    for _ in range(nrep):
        idx = np.random.randint(len(nimg_0))
        imgs_samps.append(idx)
        imgi = IMGS[idx]
        try:
            T1_ = TRNS[idx].reshape(-1,1)
            qi = QUTS[idx].tolist()
            ru = R.from_quat([qi[1],qi[2],qi[3],qi[0]]).as_euler('xyz',degrees=True)
        except:
            T1_ = np.zeros(3).reshape(-1,1)
            ru = np.zeros(3)
        R1_ = R.from_euler('xyz',ru,degrees=True).as_matrix()
        rv = np.random.uniform(0.0,1.0,1)
        if rv < 0.35:
            scale = np.random.uniform(-10.0,10.0,2)
            trans = np.random.uniform(-10.0,10.0,2)
            angv = np.random.uniform(-1.0, 1.0, 3)
        elif (rv > 0.35) and (rv < 0.5):
            scale = np.random.uniform(-80.0, 80.0,2)
            trans = np.random.uniform(-80.0, 80.0,2)
            angv = np.random.uniform(-5.0, 5.0, 3)
        else:
            scale = np.random.uniform(-180.0, 180.0,2)
            trans = np.random.uniform(-120.0, 120.0,2)
            angv = np.random.uniform(-30.0, 30.0, 3)

        R2 = R.from_euler('xyz', angv+ru, degrees=True).as_matrix()
        q2g = np.array(R.from_matrix(R2).as_quat())
        q2g = (q2g/np.linalg.norm(q2g)).tolist()
        R_c1_c2 = R2@R1_.T
        angdd = np.array(R.from_matrix(R_c1_c2).as_quat())
        angdd = angdd/np.linalg.norm(angdd)
        q_d = np.array([angdd[-1], angdd[0], angdd[1], angdd[2]])
        params = (scale, trans)
        K2 = np.array(K1_)
        out1 = comb_imgs2(pid, idx, imgi, T1_, R1_, params, K1, K2, R1_, R2, sz=sz, homo=homo, viz=viz)
        if out1 is not None:
            img12_tsr_lst, img12_viz, ptz_opti_data, m_prms = out1
            if len(img12_tsr_lst)>=1:
                for kl in range(len(img12_tsr_lst)):
                    sfxy = np.array(m_prms[kl][0:2]).flatten()
                    scxy = np.array(m_prms[kl][2:]).flatten()
                    nfs_ = scxy * (ofs + params[0])
                    nocs_ = scxy * (ocs + sfxy + params[1])
                    scale_ = nfs_ - ofs
                    trans_ = nocs_ - ocs 
                    dat_v = np.hstack((scale_, trans_, q_d))
                    all_dat.append(dat_v.flatten().tolist())
                    img12_tsr_all.append(img12_tsr_lst[kl])
                    opt_data_all.append(ptz_opti_data[kl])
                    if viz:
                        vizxx.append(img12_viz[kl])
                        
    if len(img12_tsr_all)>=1:
        iddx = np.arange(len(all_dat))
        np.random.shuffle(iddx) 
        iddx = iddx.tolist()
        print("len_all_dat: ", len(all_dat))  
        all_dat_tsr = torch.tensor(all_dat).float()[iddx,:]
        img12_tsr_ls = torch.stack(img12_tsr_all,0).float()[iddx,:,:,:]
        opt_data_all_ = [opt_data_all[nb] for nb in iddx]
    else:
        all_dat_tsr, img12_tsr_ls, opt_data_all_ = [], [], []
    return img12_tsr_ls, all_dat_tsr, opt_data_all_, vizxx


In [27]:
def update_k(fs, cs, Kn):
    Kn = np.array(Kn)   
    Kn[0,0] = fs[0] + Kn[0,0]
    Kn[1,1] = fs[1] + Kn[1,1]
    Kn[0,2] = cs[0] + Kn[0,2]
    Kn[1,2] = cs[1] + Kn[1,2]
    return Kn

In [28]:
def calRfq(qn):
    Rm = np.array(R.from_quat([qn[1],qn[2],qn[3],qn[0]]).as_matrix())
    return Rm

In [29]:
def homo_unwarp(img, Hn, w=640, h=480):
    img = np.array(img).astype(np.uint8)
    Hinv = np.linalg.inv(Hn)
    Hinv /= Hinv[2, 2]
    rotated_image = cv2.warpPerspective(img, Hinv, (w, h))
    return rotated_image

In [30]:
def comp_img_valid(u2_p_, ww=640, hh=480):
    u2_p = np.array(u2_p_)
    fl = (u2_p[0,:]>=0) * (u2_p[0,:]<ww) * (u2_p[1,:]>=0) * (u2_p[1,:]<hh)
    u2_pp = u2_p[:,fl]
    if u2_pp.shape[1]<10:
        return False
    else:
        dww = np.max(u2_pp[0,:]) - np.min(u2_pp[0,:])
        dhh = np.max(u2_pp[1,:]) - np.min(u2_pp[1,:])
        if dww>=50 and dhh>=50:
            return True
        else:
            return False

In [101]:
def eval_patch_algn(modn, pid, Kmm, iters=5, sz=300, viz=False, nnmax=5000):
    modd = modn.eval()
    with torch.no_grad():
        idx = np.random.randint(len(nimg_0))
        print("img_indx: ", nimg_0[idx])
        img1 = IMGS[idx]
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY).astype(np.uint8) 
        
        try:
            t1 = TRNS[idx].reshape(-1,1)
            q1 = QUTS[idx].tolist()
            R1_ = np.array(R.from_quat([q1[1],q1[2],q1[3],q1[0]]).as_matrix())
            ru1 = np.array(R.from_matrix(R1_).as_euler('xyz',degrees=True))
        except:
            t1 = np.zeros(3)
            ru1 = np.zeros(3)
            R1_ = np.array(R.from_euler('xyz', ru1, degrees=True).as_matrix())
        
        rv = np.random.uniform(0.0,1.0,1)
        if rv < 0.3:
            gt_scale = np.random.uniform(-5.0,5.0,2)
            gt_trans = np.random.uniform(-5.0,5.0,2)
            angv = np.random.uniform(-5.0, 5.0, 3)
        elif (rv > 0.3) and (rv < 0.6):
            gt_scale = np.random.uniform(-100.0, 100.0,2)
            gt_trans = np.random.uniform(-100.0, 100.0,2)
            angv = np.random.uniform(-15.0, 15.0, 3)
        else:
            gt_scale = np.random.uniform(-200,200,2)
            gt_trans = np.random.uniform(-200,200,2)
            shear = np.random.uniform(-200.0,200.0,2)
            angv = np.random.uniform(-30.0, 30.0, 3)
        
        R2_ = np.array(R.from_euler('xyz', ru1+angv, degrees=True).as_matrix())
        params = (gt_scale, gt_trans)
        K1 = np.array(Kmm)
        K2 = np.array(Kmm)

        hhn, wwn = img1.shape[0:2]
        ascores, a_imgs = [], []
        p_imgs = []   
        bx_smb = BXMs[idx][pid]
        ptz_ids = 100*torch.eye(4).float()
        for kk, ptz in enumerate([bx_smb]): 
            if ptz is not None:
                uv_ = ptz.T
                img1_0 = comp_mskk(img1, uv_.astype(np.int16))
                rmd = random.choice([True,False])
                img2_0, _, _, H_o, m0_prms = construct_homo(img1_0, K1, K2, R1_, R2_, params, dft=rmd)
                ones_ = np.ones_like(uv_[0,:])
                uv_1 = np.vstack((uv_,ones_))
                uv_2 = H_o @ uv_1
                uv_2 = uv_2[0:2,:]/uv_2[2:,:]
                flg1 = fratio(np.array(uv_), uv_2, th=0.45)
                if flg1:
                    img2_0 = np.array(img2_0).astype(np.uint8) 
                    ones_ = np.ones_like(uv_[0,:])
                    uv_1 = np.vstack((uv_,ones_))
                    uv_2 = H_o @ uv_1
                    uv_2 = uv_2[0:2,:]/uv_2[2:,:]
                    scores_ind = []
                    scores = []
                    imgs_aln = []
                    img1_00, img2_00 = np.array(img1_0).astype(np.uint8), np.array(img2_0).astype(np.uint8)
                    img_tsr10 = F.interpolate(torch.tensor(img1_00/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                    mskk = np.zeros((hhn,wwn),dtype=np.uint8)
                    pts = np.array(uv_).T.astype(np.int32)
                    cv2.fillPoly(mskk, pts[np.newaxis,:,:], 1)
                    vv_1n, uu_1n = np.where(mskk > 0)
                    piddx = random.sample(range(len(vv_1n)),nnmax)
                    vv_1, uu_1 = vv_1n[piddx], uu_1n[piddx]
                    ones_1 = np.ones_like(vv_1)
                    u11_i = np.vstack((uu_1, vv_1, ones_1))
                    u21_i = H_o @ u11_i
                    u21_i = u21_i/u21_i[2:,:]
                    u1_i, u2_i = u11_i[0:2,:], u21_i[0:2,:]
                    u22_i = np.vstack((u2_i, ones_1))

                    H12_c = np.eye(3)
                    for j in range(iters):
                        img_tsr1 = F.interpolate(torch.tensor(img1_0/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                        img_tsr2 = F.interpolate(torch.tensor(img2_0/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                        comb_tsr1 = torch.stack([img_tsr1, img_tsr2],0).float()
                        comb_tsr_btz = comb_tsr1.unsqueeze(0)
                        pred_ptz = modd(comb_tsr_btz.to(device))
                        fscale, cscale, quat = pred_ptz
                        fs, cs  = fscale.cpu().numpy(), cscale.cpu().numpy()
                        sh = np.zeros((2,2))
                        qtn = quat/quat.norm(p=2,dim=1,keepdim=True)
                        qtn = qtn.cpu().numpy()
                        
                        K12 = update_k(fs[0,:], cs[0,:], K1)
                        R12 = calRfq(qtn[0,:].flatten().tolist())
                        H12 = K12 @ R12 @ np.linalg.inv(K1)
                        H12_c = np.array(H12 @ H12_c)
                        img2_0 = homo_unwarp(img2_00, np.array(H12_c))
                        u2_p = H12_c @ u11_i
                        u2_p = u2_p[0:2,:]/u2_p[2:,:]
                        if not(comp_img_valid(u2_p)):
                            break
                        d12 = np.mean(np.abs(u2_i-u2_p))
                        scores.append(d12)
                        scores_ind.append(d12)
                        imgs_aln.append((img2_0, img1_0))
                    if len(scores)>=1:
                        b_ind = np.argmin(scores)
                        b_score = np.min(scores)
                        print("b_score: ", b_score, ' pixels')
                        img1_00_ = cv2.resize(img1_00,(sz,sz),interpolation=cv2.INTER_LINEAR)
                        img2_pp = cv2.resize(imgs_aln[b_ind][0],(sz,sz),interpolation=cv2.INTER_LINEAR) #cv2.resize(imgs_aln[b_ind][0],(sz,sz),interpolation=cv2.INTER_LINEAR)
                        img2_00_p = cv2.resize(img2_00,(sz,sz),interpolation=cv2.INTER_LINEAR)
                        
                        cv2.putText(img1_00_, 'Src', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        cv2.putText(img2_00_p, 'Trg.', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        cv2.putText(img2_pp, 'Rect. Trg.', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                        img1_comb = np.hstack((img1_00_, img2_pp, img2_00_p)) #
                        ascores.append(b_score) 
                        a_imgs.append(img1_comb)
                    else:
                        ascores.append(None)
                else:
                    ascores.append(None)
        else:
            ascores.append(None)
        clr = True
        if viz:
            if len(a_imgs)>=1:
                clr = False  
                clear_output(wait=True)
                aimgs = np.vstack(a_imgs)
                plt.figure(figsize=(25,10))
                plt.imshow(aimgs, cmap='gray')
                plt.show()
                     
        return ascores[0], clr

In [32]:
def typplot(gts, preds, dims, ceps, title, mk):
    plt.figure(figsize=(18, 5))
    for i in range(gts.shape[1]):
        plt.subplot(1,gts.shape[1],i+1)
        plt.plot(gts[:,i], '--*', label='GT_'+mk+' '+str(dims[i]))
        plt.plot(preds[:,i], '--*', label='pred_'+mk+' '+str(dims[i])+'\nCEP: '+str(round(ceps[i],3)))
        plt.legend()
    plt.suptitle(title)

In [98]:
def plot_progress_homo(comb_loss, preds, gts, hscores, clr):  

    if clr:
        clear_output(wait=True)
    plt.figure(figsize=(6, 3))
    plt.plot(hscores)
    plt.suptitle('Patches pixel misalignment')
    ceps = np.median(np.abs(preds - gts), 0)
    plt.figure(figsize=(18, 5))
    for i in range(len(comb_loss[0])): #5
        plt.subplot(1, len(comb_loss[0]), i+1)
        l_i = []
        for loss_k in comb_loss:
            if loss_k[i] is not None:
                l_i.append(loss_k[i])
        if len(l_i)>1:
            tag =  'loss_' 
            plt.plot(l_i, label=tag+str(i+1)+'\nCur. loss: '+
                    str(round(l_i[-1],5)))
            plt.xlabel('Epoch')
            plt.legend()
    plt.suptitle('param losses.')
    dims = ['X', 'Y', 'Z']
    typplot(gts[:,0:2], preds[:,0:2], dims, ceps[0:2], 'Scale focal', 'sf')
    typplot(gts[:,2:4], preds[:,2:4], dims, ceps[2:4], 'Scale img. centers', 'sc')
    typplot(gts[:,4:], preds[:,4:], dims, ceps[4:], 'Rotation transformation plots', 'rot')
    plt.show()
    

In [34]:
homo_models = './PATCH_HOMO_MODELS/'
if not os.path.exists(homo_models):
    os.makedirs(homo_models)

In [ ]:
### Fit and evaluate Homography Alignment Network for each patch for all images in the collection
for pid in range(n_patches):
    prev_error = 100000
    hscomb = [100,100]
    Nmax = 16 #Select according to GPU capacity
    Nepoch = 600
    Ne_max = 3500 # Set the maximum number of training episode 
    mod1 = mod1.train().to(device)
    optz_pid = optim.Adam(mod1.parameters(), lr=1e-4)
    done = False
    epoch = 0
    comb_rel_quat, gt_rel_quat, comb_loss = [], [], []
    hscores = [100, 100]
    while not(done):
        print("current_patch: ", pid)
        out3 = online_datagen(pid, K_mm, sz=300, viz=False, nrep=10, homo=True)
        if len(out3[0])>=5:
            p_gt = (out3[0][0:Nmax,:,:,:], out3[1][0:Nmax,:])
            out_preds = None
            if len(p_gt[0]>1): 
                out_preds = training_homo(p_gt, optz_pid, criterion1, mod1) 
            if out_preds is not None:
                comb_rel_quat.append(out_preds[0][0])
                gt_rel_quat.append(out_preds[0][1])
                comb_loss.append(out_preds[1])
                mme_pixels = None  
                preds, gts = np.vstack(comb_rel_quat), np.vstack(gt_rel_quat)
                if epoch%20==0:
                    vizb = True
                else:
                    vizb = False
                out4 = eval_patch_algn(mod1, pid, K_mm, iters=10, sz=300, viz=vizb)
                if out4[0] is not None:
                    hscores.append(out4[0])

                mme_pixels_ = np.median(hscomb)
                print("mean error homo: ", mme_pixels_, ' pixels')
                if epoch%20==0:
                    out_3 = plot_progress_homo(comb_loss, preds,  gts, hscores, out4[-1])
                    cum_quat = out_3   
                if mme_pixels_ is not None:
                    if mme_pixels_<=prev_error and epoch>=50:
                        torch.save(mod1.state_dict(), homo_models+'/homo_match_comb_' +str(pid)+ '.pth')
                        prev_error = mme_pixels_
                    if (epoch>=Nepoch) and (prev_error<=4.5):
                        done = True
                    elif epoch>=Ne_max:
                        done = True
            epoch += 1

In [121]:
def plot_progress_opt(a_uv_pd, a_uv_gt, comb_loss):  
    clear_output(wait=True)   
    plt.figure(figsize=(6, 3))
    for i in range(len(comb_loss[0])): #5
        plt.subplot(1, len(comb_loss[0]), i+1)
        l_i = []
        for loss_k in comb_loss:
            if loss_k[i] is not None:
                l_i.append(loss_k[i])
        
        if len(l_i)>1:
            tag =  'loss_' 
            plt.plot(l_i, label=tag+str(i+1)+'\nCur. loss: '+
                    str(round(l_i[-1],5)))
            plt.xlabel('Epoch')
            plt.legend()
    plt.suptitle('loss history')
    if a_uv_pd is not None:
        cep_u = np.round(np.median(np.abs(a_uv_gt - a_uv_pd),0),2)
        plt.figure(figsize=(10, 4))
        plt.subplot(1,2,1)
        plt.plot(a_uv_gt[:,0], '*-', label='GT')
        plt.plot(a_uv_pd[:,0], '--', label='pred_u'+'\nCEP: '+str(cep_u[0])+' pix.')
        plt.legend()
        
        plt.subplot(1,2,2)
        plt.plot(a_uv_gt[:,1], '*-', label='GT')
        plt.plot(a_uv_pd[:,1], '--', label='pred_v'+'\nCEP: '+str(cep_u[1])+' pix.')
        plt.legend()
    plt.show()
    

In [106]:
opt_models = './PATCH_OPT_MODELS/'
if not os.path.exists(opt_models):
    os.makedirs(opt_models)

In [110]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
## Finetune RAFT optical flow for pixel-wise matching

for pid in range(n_patches):
    prev_error = 100000
    Nmax = 12
    Nepoch = 600
    Ne_max = 2000
    mod2 = mod2.train().to(device)
    optimz = optim.Adam([{'params': mod2.parameters(), 'lr': 1e-4}])
    done = False
    epoch = 0
    uv_pds, uv_gts = [], []
    comb_loss = []
    while not(done):
        print("current_patch: ", pid)
        out3 = online_datagen(pid, K_mm, sz=300, viz=False, nrep=40)
        if len(out3[0])>=Nmax:
            p_gt = (out3[0][0:Nmax,:,:,:], out3[2][0:Nmax])
            if len(p_gt[0]>1):
                out_preds = training_opt(p_gt, optimz, criterion1, mod2, iters=5)
            if out_preds is not None:
                mme_pixels = None  
                if len(out_preds[0][0])>=1:
                    uv_pds.append(np.vstack(out_preds[0][0])) 
                    uv_gts.append(np.vstack(out_preds[0][1]))
                    a_uv_pd = np.vstack(uv_pds)
                    a_uv_gt = np.vstack(uv_gts)
                    mme_pixels = np.median(np.abs(a_uv_gt-a_uv_pd))
                    print("mean error: ", mme_pixels, ' pixels')
                    
                comb_loss.append(out_preds[1])
                if epoch%20==0:
                    vizb = True
                    pp = True
                else:
                    vizb = False

                if epoch%20==0:
                    out_4 =  plot_progress_opt(a_uv_pd, a_uv_gt, comb_loss)

                if mme_pixels is not None:
                    if mme_pixels<=prev_error:
                        torch.save(mod2.state_dict(), opt_models+'/raft_match_comb_' +str(pid)+ '.pth')
                        prev_error = mme_pixels
                    if (epoch>=Nepoch) and (mme_pixels<=0.3):
                        done = True
                    elif epoch>=Ne_max:
                        done = True
            epoch += 1

In [126]:
### 3d Reconstruction

In [127]:
from matplotlib.path import Path

In [128]:
from skimage.feature import hog
from sklearn.metrics.pairwise import cosine_similarity
def hog_similarity(img1, img2):
    img1 = np.array(img1).astype(np.uint8)
    img2 = np.array(img2).astype(np.uint8)
    feat1 = hog(img1, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                orientations=9, block_norm='L2-Hys', feature_vector=True)
    feat2 = hog(img2, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
                orientations=9, block_norm='L2-Hys', feature_vector=True)
    sim = cosine_similarity([feat1], [feat2])[0, 0]
    sim = (sim + 1) / 2.0
    return sim

In [129]:
def update_uv(pred_ptz_1_2, uvc_1, H12_c, polygon_path):
    p_duvc_1_2 = pred_ptz_1_2.squeeze(0).permute(1, 2, 0).cpu().numpy() 
    u_ind, v_ind = uvc_1[:,0].astype(np.int16), uvc_1[:,1].astype(np.int16)
    p_uv2_0 = uvc_1 + p_duvc_1_2[v_ind,u_ind,:]
    p_uv2_ = p_uv2_0.T
    onsx = np.ones_like(p_uv2_[0,:])
    p_uv2_ = np.vstack((p_uv2_,onsx))
    p_uv2_ = H12_c@p_uv2_
    p_uv2_ = p_uv2_[0:2,:] / (p_uv2_[2:,:] + 1e-12)
    p_uv2_ = p_uv2_.T
    p_uv2_n = np.array(p_uv2_)
    msk = polygon_path.contains_points(p_uv2_n)
    return p_uv2_, msk

In [143]:
import copy
mod11 = copy.deepcopy(mod2).eval()
mod12 = copy.deepcopy(mod2).eval()
mod21 = copy.deepcopy(mod1).eval()
mod22 = copy.deepcopy(mod1).eval()

In [133]:
def create_data_matrix(UV, Kmm, tm, Rm):
    Q = Kmm @ Rm @ tm.reshape(-1,1)
    D = Kmm @ Rm
    B = Q[0:2,:] - Q[-1,:] * UV
    A = D[0:2,:] - D[-1,:] * UV
    return A, B

In [134]:
def uv_temp(Km,t2,R2_,xyz_v=None,bxm=None,nsamp=50000):
    if (xyz_v is None) and (bxm is not None):
        x_m = np.random.uniform(bxm[:,0].min(),bxm[:,0].max(),nsamp)
        y_m = np.random.uniform(bxm[:,1].min(),bxm[:,1].max(),nsamp)
        z_m = np.random.uniform(bxm[:,2].min(),bxm[:,2].max(),nsamp)
        xyz_v = np.vstack((x_m,y_m,z_m))
    zuv2 = Km @ R2_ @ (xyz_v - t2)
    uv2 = zuv2[0:2,:]/zuv2[2:,:]
    return uv2.T, xyz_v.T

In [135]:
def ptz_uv(uv_, hhn=480,wwn=640):
    mskk = np.zeros((hhn,wwn),dtype=np.uint8)
    pts = np.array(uv_).T.astype(np.int32)
    cv2.fillPoly(mskk, pts[np.newaxis,:,:], 1)
    vv_1n, uu_1n = np.where(mskk > 0)
    uv_op = np.vstack((uu_1n, vv_1n)).T
    return uv_op

In [136]:
def get_dmin(H12,u1p,u2g):
    ons = np.ones_like(u1p[:,0]).reshape(-1,1)
    u1p = np.hstack((u1p,ons)).T
    u1p_ = H12 @ u1p
    u1p_ = u1p_[0:2,:]/u1p_[2:,:]
    dd = np.median(np.abs(u1p_.T - u2g))
    return dd

In [154]:

def calib_img(modd, modd2, im_indx, Kmm, iters=1, sz=300, n_avg=5, viz=False, init_calib=False, idd=None):
    ###This function is used for pixel-wise association and calibration of images without intrinsics according to Algorithm2
    clear_output(wait=True)
    print("im_indx: ", im_indx)
    with torch.no_grad():
        idx = nimg_0.index(im_indx[0])
        idx2 = nimg_0.index(im_indx[1])
        img1 = IMGS[idx]
        img2 = IMGS[idx2]
        imgp1, imgp2 = np.array(img1).astype(np.uint8), np.array(img2).astype(np.uint8)
        imgp1_r, imgp2_r = imgp1.astype(np.uint8), imgp2.astype(np.uint8)

        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY).astype(np.uint8)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY).astype(np.uint8)
        hh,ww = img1.shape[0:2]

        t1 = np.array(TRNS[idx]).reshape(-1,1)
        q1 = np.array(QUTS[idx]).tolist()
        R1_ = np.array(R.from_quat([q1[1],q1[2],q1[3],q1[0]]).as_matrix())
        t2 = np.array(TRNS[idx2]).reshape(-1,1)
        q2 = np.array(QUTS[idx2]).tolist()
        R2_ = np.array(R.from_quat([q2[1],q2[2],q2[3],q2[0]]).as_matrix())
        a_uv1, a_uv2, a_uv1_w = [], [], []
        a_xyz = []
        bx_smb1 = BXMs[idx]
        bx_smb2 = BXMs[idx2]
        ptz_img1, ptz_img2 = [], []
        
        for kk, (ptz1, ptz2) in enumerate(zip(bx_smb1,bx_smb2)):
            if (ptz1 is not None) and (ptz2 is not None):

                #TODO
                # Sample 3D points given bounds of the BIM to calibrate img1 - given img2 parameters
                uv2_all, XYZ_ = uv_temp(Kmm, t2, R2_, bxm=bx_smb[kk])
                uv_ = ptz1.T
                uv_2 = ptz2.T
                
                polygon_path1 = Path(uv_.T.astype(np.int16))
                polygon_path2 = Path(uv_2.T.astype(np.int16))
                
                img1_0 = comp_mskk(img1, uv_.astype(np.int16))
                img2_0 = comp_mskk(img2, uv_2.astype(np.int16))

                imgp1_r = comp_mskk(imgp1, uv_.astype(np.int16))
                imgp2_r = comp_mskk(imgp2, uv_2.astype(np.int16))
                image1 = torch.tensor(imgp1_r).permute(2, 0, 1).unsqueeze(0).cuda()
                image2 = torch.tensor(imgp2_r).permute(2, 0, 1).unsqueeze(0).cuda()
                ptz_img1.append(img1_0), ptz_img2.append(img2_0)
                
                uvc_1 = ptz_uv(uv_)
                if init_calib: ## Calibrate img1 - given img2 parameters
                    flg2 = (uv2_all[:,0]>=0) * (uv2_all[:,0]<640) * (uv2_all[:,1]>=0) * (uv2_all[:,1]<480)
                    uvc_2_kpt = uv2_all[flg2,:]
                    XYZ_ptz = XYZ_[flg2,:]
                else:
                    uvc_2_kpt = ptz_uv(uv_2)

                uvc_1_t = torch.tensor(uvc_1).float()
                uvc_2_t = torch.tensor(uvc_2_kpt).float()
                img1_00, img2_00 = np.array(img1_0).astype(np.uint8), np.array(img2_0).astype(np.uint8)
                img_tsr10 = F.interpolate(torch.tensor(img1_00/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                img_tsr20 = F.interpolate(torch.tensor(img2_00/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                K1 = np.array(Kmm)
                H12_c, H21_c = np.eye(3), np.eye(3)
                ref_uv2 = []
                ref_uv1 = []
                mtx1, mtx2, msk1, msk2 = [], [], [], []
                for j in range(iters): 
                    img_tsr1 = F.interpolate(torch.tensor(img1_0/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                    img_tsr2 = F.interpolate(torch.tensor(img2_0/255.0).unsqueeze(0).unsqueeze(0), size=(sz,sz), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                    comb_tsr1 = torch.stack([img_tsr10, img_tsr2],0).float()
                    comb_tsr2 = torch.stack([img_tsr20, img_tsr1],0).float()

                    pred_homo1 = modd2[kk](comb_tsr1.unsqueeze(0).to(device))
                    pred_homo2 = modd2[kk](comb_tsr2.unsqueeze(0).to(device))
                    
                    fscale1, cscale1, quat1 = pred_homo1
                    fs1, cs1  = fscale1.cpu().numpy(), cscale1.cpu().numpy()
                    sh = np.zeros((2,2))
                    qtn1 = quat1/quat1.norm(p=2,dim=1,keepdim=True)
                    qtn1 = qtn1.cpu().numpy()
                    
                    fscale2, cscale2, quat2 = pred_homo2
                    fs2, cs2  = fscale2.cpu().numpy(), cscale2.cpu().numpy()
                    qtn2 = quat2/quat2.norm(p=2,dim=1,keepdim=True)
                    qtn2 = qtn2.cpu().numpy()

                    K12 = update_k(fs1[0,:], cs1[0,:], K1)
                    K21 = update_k(fs2[0,:], cs2[0,:], K1) 
                    R12 = calRfq(qtn1[0,:].flatten().tolist())
                    R21 = calRfq(qtn2[0,:].flatten().tolist()) 
                    
                    H12 = K12 @ R12 @ np.linalg.inv(K1)
                    H21 = K21 @ R21 @ np.linalg.inv(K1)
                    H12_c = np.array(H12 @ H12_c)
                    H21_c = np.array(H21 @ H21_c)

                    img2_0 = homo_unwarp(img2_00, H12_c)
                    img1_0 = homo_unwarp(img1_00, H21_c)
                    
                    img2_0r = homo_unwarp(imgp2_r, H12_c)
                    img1_0r = homo_unwarp(imgp1_r, H21_c)

                    image1n = torch.tensor(img1_0r).permute(2, 0, 1).unsqueeze(0).cuda()
                    image2n = torch.tensor(img2_0r).permute(2, 0, 1).unsqueeze(0).cuda()
                    _, pred_ptz_1_2 = modd[kk](image1, image2n, iters=12, test_mode=True)
                    _, pred_ptz_2_1 = modd[kk](image2, image1n, iters=12, test_mode=True)

                    p_uv1_0, msk10 = update_uv(pred_ptz_2_1, uvc_2_kpt, H21_c, polygon_path2)
                    p_uv2_0, msk20 = update_uv(pred_ptz_1_2, uvc_1, H12_c, polygon_path1)
                    
                    # TODO
                    # Geometric metric can be used after image pairs are calibrated
                    # d10_ = get_dmin(H12_c,uv1_all,uv2_all)
                    # d12 = get_dmin(np.linalg.inv(H21_c),uv1_all,uv2_all)
                    # d20_ = get_dmin(H21_c,uv2_all,uv1_all)
                    # d21 = get_dmin(np.linalg.inv(H12_c),uv2_all,uv1_all)
                    # d10 = min([d10_, d12])
                    # d20 = min([d20_, d21])

                    #Photometric metric - applies irrespective of calibration
                    mtrc10 = hog_similarity(img2_00, img1_0)
                    mtrc20 = hog_similarity(img1_00, img2_0)
                    mtx1.append(mtrc10) #mtx1.append(d10)
                    msk1.append(msk10)
                    ref_uv1.append(p_uv1_0)
                    mtx2.append(mtrc20) #mtx2.append(d20)
                    msk2.append(msk20)
                    ref_uv2.append(p_uv2_0)
                    
        
                b_ind1 = np.argmax(mtx1) #np.argmin(mtx1) #
                b_ind2 = np.argmax(mtx2) #np.argmin(mtx2) #
                ref_uv1_arr_m = ref_uv1[b_ind1]
                ref_uv2_arr_m = ref_uv2[b_ind2]
                ff1 = (ref_uv1_arr_m[:,0]>=0) * (ref_uv1_arr_m[:,0]<ww) * (ref_uv1_arr_m[:,1]>=0) * (ref_uv1_arr_m[:,1]<hh)
                ff2 = (ref_uv2_arr_m[:,0]>=0) * (ref_uv2_arr_m[:,0]<ww) * (ref_uv2_arr_m[:,1]>=0) * (ref_uv2_arr_m[:,1]<hh)
                if init_calib:
                    a_uv1.append(ref_uv1_arr_m[ff1,:]) 
                    a_uv2.append(uvc_2_kpt[ff1,:]) 
                    a_xyz.append(XYZ_ptz[ff1,:]) 
                else:
                    a_uv1.append((ref_uv1_arr_m[ff1,:].astype(np.int16),uvc_1[ff2,:]))
                    a_uv2.append((ref_uv2_arr_m[ff2,:].astype(np.int16),uvc_2_kpt[ff1,:]))
                
            else:
                if not init_calib:
                    a_uv1.append(None)
                    a_uv2.append(None)
                    ptz_img1.append(None), ptz_img2.append(None)
        
        if len(a_uv1)>=1 and init_calib:
            ## Running PnP to calibrate img1
            AXYZ = np.vstack(a_xyz)
            AUV1 = np.vstack(a_uv1)
            AUV2 = np.vstack(a_uv2)
            if viz:
                h1, w1 = imgp1.shape[:2]; h2, w2 = imgp2.shape[:2]
                imgc = np.hstack((img1_0, img2_00))
                vis = np.zeros((max(h1,h2), w1+w2, 3), dtype=np.uint8)
                vis[:h1, :w1, :] = np.array(imgp1).astype(np.uint8)
                vis[:h2, w1:w1+w2, :] = np.array(imgp2).astype(np.uint8)
                vis2 = np.array(vis).astype(np.uint8)
                
                pts1 = np.array(AUV1)
                pts2 = np.array(AUV2)
                nnmax = 50
                nnid = random.sample(range(len(pts1)),nnmax)
                pts1_ = pts1[nnid,:]
                pts2_ = pts2[nnid,:]
                color = (0,255,0)
                for (x1,y1), (x2,y2) in zip(pts1_, pts2_):
                    pt1 = (int(x1), int(y1))
                    pt2 = (int(x2)+w1, int(y2))
                    cv2.line(vis, pt1, pt2, color, 1, cv2.LINE_AA)
                    cv2.circle(vis, pt1, 4, color, -1)
                    cv2.circle(vis, pt2, 4, color, -1)
                plt.figure(figsize=(12,12))
                plt.imshow(vis) 
                
                ## Evolution of HOG similarity scores
                plt.figure(figsize=(7,3))
                plt.plot(mtx1, '--*',label='metrics_1_2')
                plt.plot(mtx2,'--*',label='metrics_2_1')
                plt.legend()
                plt.show()
                plt.close()
            
            
            print("AXYZ: ", AXYZ.shape)
            print("AUV1: ", AUV1.shape)
            dist_coeffs = np.zeros(5)
            if len(AUV1>=8):
                success, rvec, tvec, inliers = cv2.solvePnPRansac(
                    AXYZ.astype(np.float32), AUV1.astype(np.float32), Kmm, dist_coeffs,
                    reprojectionError=20.0,
                    confidence=0.99,
                    flags=cv2.SOLVEPNP_ITERATIVE
                )
                if success:
                    Rm, _ = cv2.Rodrigues(rvec)
                    trans = (-Rm.T@tvec).flatten()
                    TRNS[idx] = np.array(trans).flatten()
                    R_idx = np.array(Rm)
                    q_idx = np.array(R.from_matrix(R_idx).as_quat()).tolist()
                    QUTS[idx] = [q_idx[-1],q_idx[0],q_idx[1],q_idx[2]]
        else:
            return a_uv1, a_uv2, ptz_img1, ptz_img2

In [165]:
nimg_0s = nimg_0[:]
nimg_gs = list(itertools.combinations(nimg_0s, 2)) ## Form pairs of images in the collection 
idx1, idx2 = 0, 5 # Example to calibrate nimg_0s[0], given parematers of nimg_0s[5]

In [ ]:
mods1 = []
mods2 = []
for pk in range(n_patches):
    mod12.load_state_dict(torch.load(opt_models+'/raft_match_comb_' +str(pk)+ '.pth'))
    mod22.load_state_dict(torch.load(homo_models+'/homo_match_comb_' +str(pk)+ '.pth'))
    mod1_pk = mod12.eval()
    mod2_pk = mod22.eval()
    mods1.append(mod1_pk)
    mods2.append(mod2_pk)
    

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
im_indx = [nimg_0[idx1],nimg_0[idx2]]
out_c = calib_img(mods1, mods2, im_indx, K_mm, iters=10, sz=300, n_avg=50, viz=True, init_calib=True, idd=idx1)

In [156]:
def calc_opti_data(nimgs, nimgs_g, Kmm, iters, sz, n_avg, clib=False):
    # build correspondences for 3D reconstruction
    pts_3d = {}
    idx_hist = [{} for _ in range(len(nimg_0))]  
    next_pid = 0
    for kk, pp in enumerate(nimgs_g):
        idx1, idx2 = nimg_0.index(pp[0]), nimg_0.index(pp[1])
        im_indx = pp[:]
        out_c = calib_img(mods1, mods2, im_indx, Kmm, iters=iters, sz=sz, n_avg=n_avg, viz=False, init_calib=clib)
        a_uv1, a_uv2, _, _ = out_c
        img1_, img2_ = IMGS[idx1], IMGS[idx2]
        a_img1_kpts, a_img2_kpts = [], []
        for jj in range(n_patches): 
            if a_uv1[jj] is not None:
                img1_kpts, img2_kpts = a_uv1[jj][1], a_uv2[jj][1]
                img1_kpts_0, img2_kpts_0 = a_uv1[jj][0], a_uv2[jj][0]
                img1_kpts = np.vstack((img1_kpts_0, img1_kpts))
                img2_kpts = np.vstack((img2_kpts, img2_kpts_0))
                a_img1_kpts.append(img1_kpts), a_img2_kpts.append(img2_kpts)
        
        if len(a_img1_kpts)>=1:
            img1_kpts = np.vstack(a_img1_kpts)
            img2_kpts = np.vstack(a_img2_kpts)
            u1_idx = np.arange(img1_kpts.shape[0])
            u2_idx = np.arange(img2_kpts.shape[0])

            
            for ii in range(len(u1_idx)):
                uv1 = tuple(np.round(img1_kpts[u1_idx[ii], :].copy(), 6))
                uv2 = tuple(np.round(img2_kpts[u2_idx[ii], :].copy(), 6))
                cc1 = np.array(uv1).astype(np.int16)
                cc2 = np.array(uv2).astype(np.int16)
                if uv1 in idx_hist[idx1]:
                    pid = idx_hist[idx1][uv1]
                    if not any(img_idx == pp[1] for _, img_idx, _ in pts_3d[str(pid)]):
                        pts_3d[str(pid)].append((np.array(uv2), nimgs.index(pp[1]), img2_[cc2[1],cc2[0],:]))
                        idx_hist[idx2][uv2] = pid

                elif uv2 in idx_hist[idx2]:
                    pid = idx_hist[idx2][uv2]
                    if not any(img_idx == pp[0] for _, img_idx, _ in pts_3d[str(pid)]):
                        pts_3d[str(pid)].append((np.array(uv1), nimgs.index(pp[0]), img1_[cc1[1],cc1[0],:]))
                        idx_hist[idx1][uv1] = pid

                else:
                    pid = next_pid
                    pts_3d[str(pid)] = [
                        (np.array(uv1), nimgs.index(pp[0]), img1_[cc1[1],cc1[0],:]),
                        (np.array(uv2), nimgs.index(pp[1]), img2_[cc2[1],cc2[0],:])
                    ]
                    idx_hist[idx1][uv1] = pid
                    idx_hist[idx2][uv2] = pid
                    next_pid += 1
    return pts_3d

In [164]:
## At this state all images in the collection are calibrated
TRNS_ = TRNS[:] 
QUTS_ = QUTS[:]

In [158]:
def reconstruct(pcd_dict, Kmm):
    ## Traingulate 3D points
    XYZpts = []
    pcols = []
    uv_vals = []
    for key in pcd_dict.keys():
        elems_to_xyz = pcd_dict[key]
        uv_i = []
        Am, Bm = [], []
        acols = []
        if len(elems_to_xyz)>=2:
            try:
                for elem in elems_to_xyz: 
                    uv, idx, color = elem
                    uv_i.append((uv.flatten(), idx))
                    acols.append(color)
                    tm = np.array(TRNS_[idx])
                    qm = QUTS_[idx]
                    Rm = np.array(R.from_quat([qm[1],qm[2],qm[3],qm[0]]).as_matrix())
                    am, bm = create_data_matrix(uv.reshape(-1,1), Kmm, tm, Rm)
                    Am.append(am), Bm.append(bm)

                Am, Bm = np.vstack(Am), np.vstack(Bm)
                xyz_ls = np.linalg.inv(Am.T @ Am) @ Am.T @ Bm
                XYZpts.append(xyz_ls.flatten())
                pcols.append(np.median(acols,0).astype(np.uint8))
                uv_vals.append(uv_i)
            except:
                pass
        if len(XYZpts)>=850000:
            break
    return XYZpts, pcols, uv_vals

In [163]:
## Visualize 3D reconstruction
def plot_pointcloud(XYZ, colors):
    XYZ = np.asarray(XYZ)
    colors = np.asarray(colors) / 255.0
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(
        XYZ[:, 1], XYZ[:, 0], XYZ[:, 2],
        c=colors, marker='o', s=0.9
    )
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_xlim([-18,18.0])
    ax.set_zlim([0.0,16.0])

    max_range = np.array([
        XYZ[:, 0].max() - XYZ[:, 0].min(),
        XYZ[:, 1].max() - XYZ[:, 1].min(),
        XYZ[:, 2].max() - XYZ[:, 2].min()
    ]).max() / 2.0
    # mid_x = (XYZ[:, 0].max() + XYZ[:, 0].min()) * 0.5
    # mid_y = (XYZ[:, 1].max() + XYZ[:, 1].min()) * 0.5
    # mid_z = (XYZ[:, 2].max() + XYZ[:, 2].min()) * 0.5
    # ax.set_xlim(mid_x - max_range, mid_x + max_range)
    # ax.set_ylim(mid_y - max_range, mid_y + max_range)
    # ax.set_zlim(mid_z - max_range, mid_z + max_range)
    plt.show()
    plt.close()

In [ ]:
out5 = calc_opti_data(nimg_0s, nimg_gs, K_mm, iters=10, sz=300, n_avg=50, clib=False)
print('running 3D reconstruction')
out6, pcols, uv_vals = reconstruct(out5, K_mm)
XYZpts_p = np.vstack(out6)
print(XYZpts_p.shape)
plot_pointcloud(XYZpts_p, pcols)